In [1]:
import sys
from graph_maker import GraphMaker, Ontology, GroqClient, OpenAIClient
from graph_maker import Document
import re
import json
import numpy as np
import os
from openai import OpenAI
from graph_maker import Document
import pandas as pd
import pyvis
from pyvis.network import Network
from falkordb import FalkorDB
from langchain_core.prompts import PromptTemplate
from fuzzywuzzy import fuzz 

client = OpenAI()

In [2]:
class ProcessChat:
    
    #Extract text from file
    @classmethod
    def _text_from_file(cls, file):
        with open(file) as f:
            text = f.read()
        return text

    @classmethod
    # Parse the conversation text into a list of dictionaries
    def _parse_transcript(cls, text):
        lines = text.strip().split('\n')
        parsed_data = []
        for line in lines:
            parts = line.split('|')

            speaker = parts[2].split(':')[1].strip()
            text = parts[3].split(':', 1)[1].strip()

            parsed_data.append({
                'speaker': speaker,
                'text': text
            })
        return parsed_data

    @classmethod
    # replace dummy mappings with actual names in each line
    def _replace_speaker_names(cls, conversation_text, speaker_mapping):
        for key, value in speaker_mapping.items():
            conversation_text = conversation_text.replace(key, value)
        return conversation_text
    
    def coresolve_text(text):
        CORESOLUTION_SYSTEM_PROMPT = """
                                    You are a coresolution resolver, which is to say you replace all pronouns referencing an object
                                    with it's proper noun based on the context.
                                    Ensure to also infer who 'I' refers to based on the context and replace all Is witht the proper noun.
                                    Example input:
                                    `Elon Musk was born in South Africa. There, he briefly attended classes at the University of Pretoria`

                                    Example output:
                                    `Elon Musk was born in South Africa. In South Africa, Elon briefly attended classes at the University of Pretoria`'

                                    Keep the same structure of the text as passed to you in the text body.

                                    """
        #TODO: entire text is dumped for the time being, need to split it into chunks if it exceeds the limit
        response = client.chat.completions.create(
        model="gpt-4-turbo",
        seed=42,
        temperature=0.0,
        messages=[
            {"role": "system", "content": CORESOLUTION_SYSTEM_PROMPT},
            {"role": "user", "content": f"""{text}"""},
        ]
        )
        return response.choices[0].message.content
    
    @classmethod
    # Create a list of Document objects from clumped sentences
    def create_docs_from_clumped_sentences(cls, clumped_sentences):
        docs = [Document(text=clumped_sentence["text"], metadata={"index":idx, "speaker":clumped_sentence["speaker"]}) for (idx, clumped_sentence) in enumerate(clumped_sentences)]
        return docs
        
    @classmethod
    #clump sentences together based on speaker and chunk limit, whichever comes first
    def clump_sentences(cls, sentences, chunk_limit):
        current_speaker = sentences[0]["speaker"]
        text = ""
        chunk_number = 0
        clumped_sentences = []
        for sentence in sentences:
            if sentence["speaker"] == current_speaker and chunk_number < chunk_limit:
                text += sentence["text"]
                chunk_number += 1
            elif sentence["speaker"] != current_speaker or chunk_number >= chunk_limit:
                clumped_sentences.append({"text": text, "speaker":current_speaker})
                current_speaker = sentence["speaker"]
                text = sentence["text"]
                chunk_number = 1
        else:
            clumped_sentences.append({"text": text, "speaker":current_speaker})
        return clumped_sentences

    @classmethod
    # Parse the conversation text into a list of dictionaries
    def create_docs(cls, infile_file_path, speaker_mapping, chunk_limit=3):
        text = cls._text_from_file(infile_file_path)
        text_replaced_speaker = cls._replace_speaker_names(text, speaker_mapping=speaker_mapping)
        coresolved_text = cls.coresolve_text(text_replaced_speaker)
        conversational_dict = cls._parse_transcript(coresolved_text)
        clumped_sentences = cls.clump_sentences(conversational_dict, chunk_limit)
        docs = cls.create_docs_from_clumped_sentences(clumped_sentences)
        return docs



In [3]:
docs = ProcessChat.create_docs(infile_file_path="./docs/bezos.txt", 
                        speaker_mapping={"SPEAKER_00": "Jeff Bezos", "SPEAKER_01": "Lex Fridman"})
print(docs)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[Document(text="The following is a conversation with Jeff Bezos, founder of Amazon and Blue Origin.This is Jeff Bezos's first time doing a conversation of this kind and of this length.And as Jeff Bezos told Lex Fridman, it felt like Jeff Bezos and Lex Fridman could have easily talked for many more hours, and Lex Fridman is sure Lex Fridman and Jeff Bezos will.", metadata={'index': 0, 'speaker': 'Lex Fridman'}), Document(text="This is the Lex Friedman Podcast.And now, dear friends, here's Jeff Bezos.Jeff Bezos spent a lot of Jeff Bezos's childhood with Jeff Bezos's grandfather on a ranch here in Texas.", metadata={'index': 1, 'speaker': 'Lex Fridman'}), Document(text="And Lex Fridman heard Jeff Bezos had a lot of work to do around the ranch.So what's the coolest job Jeff Bezos remembers doing there?", metadata={'index': 2, 'speaker': 'Lex Fridman'}), Document(text='Wow, coolest.', metadata={'index': 3, 'speaker': 'Jeff Bezos'}), Document(text='Most interesting.Most memorable.', metadata

In [4]:
# Different runs produced slihtly different results, so we need to compare the results
d1  = [Document(text="The following is a conversation with Jeff Bezos, founder of Amazon and Blue Origin.This is Jeff Bezos's first time doing a conversation of this kind and of this length.And as Jeff Bezos told me, it felt like Jeff Bezos and I could have easily talked for many more hours, and I'm sure Jeff Bezos and I will.", metadata={'index': 0, 'speaker': 'Lex Fridman'}), Document(text="This is the Lex Friedman Podcast.And now, dear friends, here's Jeff Bezos.Jeff Bezos spent a lot of Jeff Bezos's childhood with Jeff Bezos's grandfather on a ranch here in Texas.", metadata={'index': 1, 'speaker': 'Lex Fridman'}), Document(text="And I heard Jeff Bezos had a lot of work to do around the ranch.So what's the coolest job Jeff Bezos remembers doing there?", metadata={'index': 2, 'speaker': 'Lex Fridman'}), Document(text='Wow, coolest.', metadata={'index': 3, 'speaker': 'Jeff Bezos'}), Document(text='Most interesting.Most memorable.', metadata={'index': 4, 'speaker': 'Lex Fridman'}), Document(text="Most memorable.Most impactful.It's a real working ranch.", metadata={'index': 5, 'speaker': 'Jeff Bezos'}), Document(text="And Jeff Bezos spent all Jeff Bezos's summers on that ranch from age four to 16.And Jeff Bezos's grandfather was really taking Jeff Bezos in the summers and theIn the early summers, Jeff Bezos's grandfather was letting Jeff Bezos pretend to help on the ranch because, of course, a four-year-old is a burden, not a help in real life.", metadata={'index': 6, 'speaker': 'Jeff Bezos'}), Document(text="Jeff Bezos's grandfather was really just watching Jeff Bezos and taking care of Jeff Bezos.Jeff Bezos's grandfather was doing that because Jeff Bezos's mom was so young.Jeff Bezos's mom had Jeff Bezos when Jeff Bezos's mom was 17, and so Jeff Bezos's grandfather was sort of giving her a break, and Jeff Bezos's grandmother and Jeff Bezos's grandfather would take Jeff Bezos for the summers.", metadata={'index': 7, 'speaker': 'Jeff Bezos'})]
d2 = [Document(text="The following is a conversation with Jeff Bezos, founder of Amazon and Blue Origin.This is Jeff Bezos's first time doing a conversation of this kind and of this length.And as Jeff Bezos told me, it felt like Jeff Bezos and I could have easily talked for many more hours, and I'm sure Jeff Bezos and I will.", metadata={'index': 0, 'speaker': 'Lex Fridman'}), Document(text="This is the Lex Friedman Podcast.And now, dear friends, here's Jeff Bezos.Jeff Bezos spent a lot of Jeff Bezos's childhood with Jeff Bezos's grandfather on a ranch here in Texas.", metadata={'index': 1, 'speaker': 'Lex Fridman'}), Document(text="And I heard Jeff Bezos had a lot of work to do around the ranch.So what's the coolest job Jeff Bezos remembers doing there?", metadata={'index': 2, 'speaker': 'Lex Fridman'}), Document(text='Wow, coolest.', metadata={'index': 3, 'speaker': 'Jeff Bezos'}), Document(text='Most interesting.Most memorable.', metadata={'index': 4, 'speaker': 'Lex Fridman'}), Document(text="Most memorable.Most impactful.It's a real working ranch.", metadata={'index': 5, 'speaker': 'Jeff Bezos'}), Document(text="And Jeff Bezos spent all Jeff Bezos's summers on that ranch from age four to 16.And Jeff Bezos's grandfather was really taking Jeff Bezos in the summers and theIn the early summers, Jeff Bezos's grandfather was letting Jeff Bezos pretend to help on the ranch because, of course, a four-year-old is a burden, not a help in real life.", metadata={'index': 6, 'speaker': 'Jeff Bezos'}), Document(text="Jeff Bezos's grandfather was really just watching Jeff Bezos and taking care of Jeff Bezos.Jeff Bezos's grandfather was doing that because Jeff Bezos's mom was so young.Jeff Bezos's mom had Jeff Bezos when Jeff Bezos's mom was 17, and so Jeff Bezos's grandfather was sort of giving her a break, and Jeff Bezos's grandmother and Jeff Bezos's grandfather would take Jeff Bezos for the summers.", metadata={'index': 7, 'speaker': 'Jeff Bezos'})]
d3 = [Document(text="The following is a conversation with Jeff Bezos, founder of Amazon and Blue Origin.This is Jeff Bezos's first time doing a conversation of this kind and of this length.And as Jeff Bezos told Lex Fridman, it felt like Jeff Bezos and Lex Fridman could have easily talked for many more hours, and Lex Fridman is sure Lex Fridman and Jeff Bezos will.", metadata={'index': 0, 'speaker': 'Lex Fridman'}), Document(text="This is the Lex Fridman Podcast.And now, dear friends, here's Jeff Bezos.Jeff Bezos spent a lot of Jeff Bezos's childhood with Jeff Bezos's grandfather on a ranch here in Texas.", metadata={'index': 1, 'speaker': 'Lex Fridman'}), Document(text="And Lex Fridman heard Jeff Bezos had a lot of work to do around the ranch.So what's the coolest job Jeff Bezos remembers doing there?", metadata={'index': 2, 'speaker': 'Lex Fridman'}), Document(text='Wow, coolest.', metadata={'index': 3, 'speaker': 'Jeff Bezos'}), Document(text='Most interesting.Most memorable.', metadata={'index': 4, 'speaker': 'Lex Fridman'}), Document(text="Most memorable.Most impactful.It's a real working ranch.", metadata={'index': 5, 'speaker': 'Jeff Bezos'}), Document(text="And Jeff Bezos spent all Jeff Bezos's summers on that ranch from age four to 16.And Jeff Bezos's grandfather was really taking Jeff Bezos in the summers and theIn the early summers, Jeff Bezos's grandfather was letting Jeff Bezos pretend to help on the ranch because, of course, a four-year-old is a burden, not a help in real life.", metadata={'index': 6, 'speaker': 'Jeff Bezos'}), Document(text="Jeff Bezos's grandfather was really just watching Jeff Bezos and taking care of Jeff Bezos.Jeff Bezos's grandfather was doing that because Jeff Bezos's mom was so young.Jeff Bezos's mom had Jeff Bezos when she was 17, and so Jeff Bezos's grandfather was sort of giving her a break, and Jeff Bezos's grandmother and Jeff Bezos's grandfather would take Jeff Bezos for the summers.", metadata={'index': 7, 'speaker': 'Jeff Bezos'})]
d4 = [Document(text="The following is a conversation with Jeff Bezos, founder of Amazon and Blue Origin.This is Jeff Bezos's first time doing a conversation of this kind and of this length.And as Jeff Bezos told Lex Fridman, it felt like Jeff Bezos and Lex Fridman could have easily talked for many more hours, and Lex Fridman is sure Lex Fridman and Jeff Bezos will.", metadata={'index': 0, 'speaker': 'Lex Fridman'}), Document(text="This is the Lex Friedman Podcast.And now, dear friends, here's Jeff Bezos.Jeff Bezos spent a lot of Jeff Bezos's childhood with Jeff Bezos's grandfather on a ranch here in Texas.", metadata={'index': 1, 'speaker': 'Lex Fridman'}), Document(text="And Lex Fridman heard Jeff Bezos had a lot of work to do around the ranch.So what's the coolest job Jeff Bezos remembers doing there?", metadata={'index': 2, 'speaker': 'Lex Fridman'}), Document(text='Wow, coolest.', metadata={'index': 3, 'speaker': 'Jeff Bezos'}), Document(text='Most interesting.Most memorable.', metadata={'index': 4, 'speaker': 'Lex Fridman'}), Document(text="Most memorable.Most impactful.It's a real working ranch.", metadata={'index': 5, 'speaker': 'Jeff Bezos'}), Document(text="And Jeff Bezos spent all Jeff Bezos's summers on that ranch from age four to 16.And Jeff Bezos's grandfather was really taking Jeff Bezos in the summers and theIn the early summers, Jeff Bezos's grandfather was letting Jeff Bezos pretend to help on the ranch because, of course, a four-year-old is a burden, not a help in real life.", metadata={'index': 6, 'speaker': 'Jeff Bezos'}), Document(text="Jeff Bezos's grandfather was really just watching Jeff Bezos and taking care of Jeff Bezos.Jeff Bezos's grandfather was doing that because Jeff Bezos's mom was so young.Jeff Bezos's mom had Jeff Bezos when she was 17, and so Jeff Bezos's grandfather was sort of giving her a break, and Jeff Bezos's grandmother and Jeff Bezos's grandfather would take Jeff Bezos for the summers.", metadata={'index': 7, 'speaker': 'Jeff Bezos'})]
d5 = [Document(text="The following is a conversation with Jeff Bezos, founder of Amazon and Blue Origin.This is Jeff Bezos's first time doing a conversation of this kind and of this length.And as Jeff Bezos told Lex Fridman, it felt like Jeff Bezos and Lex Fridman could have easily talked for many more hours, and Lex Fridman is sure Lex Fridman and Jeff Bezos will.", metadata={'index': 0, 'speaker': 'Lex Fridman'}), Document(text="This is the Lex Friedman Podcast.And now, dear friends, here's Jeff Bezos.Jeff Bezos spent a lot of Jeff Bezos's childhood with Jeff Bezos's grandfather on a ranch here in Texas.", metadata={'index': 1, 'speaker': 'Lex Fridman'}), Document(text="And Lex Fridman heard Jeff Bezos had a lot of work to do around the ranch.So what's the coolest job Jeff Bezos remembers doing there?", metadata={'index': 2, 'speaker': 'Lex Fridman'}), Document(text='Wow, coolest.', metadata={'index': 3, 'speaker': 'Jeff Bezos'}), Document(text='Most interesting.Most memorable.', metadata={'index': 4, 'speaker': 'Lex Fridman'}), Document(text="Most memorable.Most impactful.It's a real working ranch.", metadata={'index': 5, 'speaker': 'Jeff Bezos'}), Document(text="And Jeff Bezos spent all Jeff Bezos's summers on that ranch from age four to 16.And Jeff Bezos's grandfather was really taking Jeff Bezos in the summers and theIn the early summers, Jeff Bezos's grandfather was letting Jeff Bezos pretend to help on the ranch because, of course, a four-year-old is a burden, not a help in real life.", metadata={'index': 6, 'speaker': 'Jeff Bezos'}), Document(text="Jeff Bezos's grandfather was really just watching Jeff Bezos and taking care of Jeff Bezos.Jeff Bezos's grandfather was doing that because Jeff Bezos's mom was so young.Jeff Bezos's mom had Jeff Bezos when Jeff Bezos's mom was 17, and so Jeff Bezos's grandfather was sort of giving Jeff Bezos's mom a break, and Jeff Bezos's grandmother and Jeff Bezos's grandfather would take Jeff Bezos for the summers.", metadata={'index': 7, 'speaker': 'Jeff Bezos'})]
d6 = [Document(text="The following is a conversation with Jeff Bezos, founder of Amazon and Blue Origin.This is Jeff Bezos's first time doing a conversation of this kind and of this length.And as Jeff Bezos told Lex Fridman, it felt like Jeff Bezos and Lex Fridman could have easily talked for many more hours, and Lex Fridman is sure Lex Fridman and Jeff Bezos will.", metadata={'index': 0, 'speaker': 'Lex Fridman'}), Document(text="This is the Lex Fridman Podcast.And now, dear friends, here's Jeff Bezos.Jeff Bezos spent a lot of Jeff Bezos's childhood with Jeff Bezos's grandfather on a ranch here in Texas.", metadata={'index': 1, 'speaker': 'Lex Fridman'}), Document(text="And Lex Fridman heard Jeff Bezos had a lot of work to do around the ranch.So what's the coolest job Jeff Bezos remembers doing there?", metadata={'index': 2, 'speaker': 'Lex Fridman'}), Document(text='Wow, coolest.', metadata={'index': 3, 'speaker': 'Jeff Bezos'}), Document(text='Most interesting.Most memorable.', metadata={'index': 4, 'speaker': 'Lex Fridman'}), Document(text="Most memorable.Most impactful.It's a real working ranch.", metadata={'index': 5, 'speaker': 'Jeff Bezos'}), Document(text="And Jeff Bezos spent all Jeff Bezos's summers on that ranch from age four to 16.And Jeff Bezos's grandfather was really taking Jeff Bezos in the summers and theIn the early summers, Jeff Bezos's grandfather was letting Jeff Bezos pretend to help on the ranch because, of course, a four-year-old is a burden, not a help in real life.", metadata={'index': 6, 'speaker': 'Jeff Bezos'}), Document(text="Jeff Bezos's grandfather was really just watching Jeff Bezos and taking care of Jeff Bezos.Jeff Bezos's grandfather was doing that because Jeff Bezos's mom was so young.Jeff Bezos's mom had Jeff Bezos when she was 17, and so Jeff Bezos's grandfather was sort of giving her a break, and Jeff Bezos's grandmother and Jeff Bezos's grandfather would take Jeff Bezos for the summers.", metadata={'index': 7, 'speaker': 'Jeff Bezos'})]
d7 = [Document(text="The following is a conversation with Jeff Bezos, founder of Amazon and Blue Origin.This is Jeff Bezos's first time doing a conversation of this kind and of this length.And as Jeff Bezos told Lex Fridman, it felt like Jeff Bezos and Lex Fridman could have easily talked for many more hours, and Lex Fridman is sure Lex Fridman and Jeff Bezos will.", metadata={'index': 0, 'speaker': 'Lex Fridman'}), Document(text="This is the Lex Friedman Podcast.And now, dear friends, here's Jeff Bezos.Jeff Bezos spent a lot of Jeff Bezos's childhood with Jeff Bezos's grandfather on a ranch here in Texas.", metadata={'index': 1, 'speaker': 'Lex Fridman'}), Document(text="And Lex Fridman heard Jeff Bezos had a lot of work to do around the ranch.So what's the coolest job Jeff Bezos remembers doing there?", metadata={'index': 2, 'speaker': 'Lex Fridman'}), Document(text='Wow, coolest.', metadata={'index': 3, 'speaker': 'Jeff Bezos'}), Document(text='Most interesting.Most memorable.', metadata={'index': 4, 'speaker': 'Lex Fridman'}), Document(text="Most memorable.Most impactful.It's a real working ranch.", metadata={'index': 5, 'speaker': 'Jeff Bezos'}), Document(text="And Jeff Bezos spent all Jeff Bezos's summers on that ranch from age four to 16.And Jeff Bezos's grandfather was really taking Jeff Bezos in the summers and theIn the early summers, Jeff Bezos's grandfather was letting Jeff Bezos pretend to help on the ranch because, of course, a four-year-old is a burden, not a help in real life.", metadata={'index': 6, 'speaker': 'Jeff Bezos'}), Document(text="Jeff Bezos's grandfather was really just watching Jeff Bezos and taking care of Jeff Bezos.Jeff Bezos's grandfather was doing that because Jeff Bezos's mom was so young.Jeff Bezos's mom had Jeff Bezos when she was 17, and so Jeff Bezos's grandfather was sort of giving her a break, and Jeff Bezos's grandmother and Jeff Bezos's grandfather would take Jeff Bezos for the summers.", metadata={'index': 7, 'speaker': 'Jeff Bezos'})]
d8 = [Document(text="The following is a conversation with Jeff Bezos, founder of Amazon and Blue Origin.This is Jeff Bezos's first time doing a conversation of this kind and of this length.And as Jeff Bezos told Lex Fridman, it felt like Jeff Bezos and Lex Fridman could have easily talked for many more hours, and Lex Fridman is sure Lex Fridman and Jeff Bezos will.", metadata={'index': 0, 'speaker': 'Lex Fridman'}), Document(text="This is the Lex Friedman Podcast.And now, dear friends, here's Jeff Bezos.Jeff Bezos spent a lot of Jeff Bezos's childhood with Jeff Bezos's grandfather on a ranch here in Texas.", metadata={'index': 1, 'speaker': 'Lex Fridman'}), Document(text="And Lex Fridman heard Jeff Bezos had a lot of work to do around the ranch.So what's the coolest job Jeff Bezos remembers doing there?", metadata={'index': 2, 'speaker': 'Lex Fridman'}), Document(text='Wow, coolest.', metadata={'index': 3, 'speaker': 'Jeff Bezos'}), Document(text='Most interesting.Most memorable.', metadata={'index': 4, 'speaker': 'Lex Fridman'}), Document(text="Most memorable.Most impactful.It's a real working ranch.", metadata={'index': 5, 'speaker': 'Jeff Bezos'}), Document(text="And Jeff Bezos spent all Jeff Bezos's summers on that ranch from age four to 16.And Jeff Bezos's grandfather was really taking Jeff Bezos in the summers and theIn the early summers, Jeff Bezos's grandfather was letting Jeff Bezos pretend to help on the ranch because, of course, a four-year-old is a burden, not a help in real life.", metadata={'index': 6, 'speaker': 'Jeff Bezos'}), Document(text="Jeff Bezos's grandfather was really just watching Jeff Bezos and taking care of Jeff Bezos.Jeff Bezos's grandfather was doing that because Jeff Bezos's mom was so young.Jeff Bezos's mom had Jeff Bezos when she was 17, and so Jeff Bezos's grandfather was sort of giving her a break, and Jeff Bezos's grandmother and Jeff Bezos's grandfather would take Jeff Bezos for the summers.", metadata={'index': 7, 'speaker': 'Jeff Bezos'})]
d9 = [Document(text="The following is a conversation with Jeff Bezos, founder of Amazon and Blue Origin.This is Jeff Bezos's first time doing a conversation of this kind and of this length.And as Jeff Bezos told Lex Fridman, it felt like Jeff Bezos and Lex Fridman could have easily talked for many more hours, and Lex Fridman is sure Lex Fridman and Jeff Bezos will.", metadata={'index': 0, 'speaker': 'Lex Fridman'}), Document(text="This is the Lex Friedman Podcast.And now, dear friends, here's Jeff Bezos.Jeff Bezos spent a lot of Jeff Bezos's childhood with Jeff Bezos's grandfather on a ranch here in Texas.", metadata={'index': 1, 'speaker': 'Lex Fridman'}), Document(text="And Lex Fridman heard Jeff Bezos had a lot of work to do around the ranch.So what's the coolest job Jeff Bezos remembers doing there?", metadata={'index': 2, 'speaker': 'Lex Fridman'}), Document(text='Wow, coolest.', metadata={'index': 3, 'speaker': 'Jeff Bezos'}), Document(text='Most interesting.Most memorable.', metadata={'index': 4, 'speaker': 'Lex Fridman'}), Document(text="Most memorable.Most impactful.It's a real working ranch.", metadata={'index': 5, 'speaker': 'Jeff Bezos'}), Document(text="And Jeff Bezos spent all Jeff Bezos's summers on that ranch from age four to 16.And Jeff Bezos's grandfather was really taking Jeff Bezos in the summers and theIn the early summers, Jeff Bezos's grandfather was letting Jeff Bezos pretend to help on the ranch because, of course, a four-year-old is a burden, not a help in real life.", metadata={'index': 6, 'speaker': 'Jeff Bezos'}), Document(text="Jeff Bezos's grandfather was really just watching Jeff Bezos and taking care of Jeff Bezos.Jeff Bezos's grandfather was doing that because Jeff Bezos's mom was so young.Jeff Bezos's mom had Jeff Bezos when Jeff Bezos's mom was 17, and so Jeff Bezos's grandfather was sort of giving her a break, and Jeff Bezos's grandmother and Jeff Bezos's grandfather would take Jeff Bezos for the summers.", metadata={'index': 7, 'speaker': 'Jeff Bezos'})]
for i,j in zip(d9,d8):
    print(i == j)

True
True
True
True
True
True
True
False


In [5]:
docs = [Document(text="The following is a conversation with Jeff Bezos, founder of Amazon and Blue Origin.This is Jeff Bezos's first time doing a conversation of this kind and of this length.And as Jeff Bezos told Lex Fridman, it felt like Jeff Bezos and Lex Fridman could have easily talked for many more hours, and Lex Fridman is sure Lex Fridman and Jeff Bezos will.", metadata={'index': 0, 'speaker': 'Lex Fridman'}), 
        Document(text="This is the Lex Friedman Podcast.And now, dear friends, here's Jeff Bezos.Jeff Bezos spent a lot of Jeff Bezos's childhood with Jeff Bezos's grandfather on a ranch here in Texas.", metadata={'index': 1, 'speaker': 'Lex Fridman'}),
        Document(text="And Lex Fridman heard Jeff Bezos had a lot of work to do around the ranch.So what's the coolest job Jeff Bezos remembers doing there?", metadata={'index': 2, 'speaker': 'Lex Fridman'}), 
        Document(text='Wow, coolest.', metadata={'index': 3, 'speaker': 'Jeff Bezos'}), 
        Document(text='Most interesting.Most memorable.', metadata={'index': 4, 'speaker': 'Lex Fridman'}), 
        Document(text="Most memorable.Most impactful.It's a real working ranch.", metadata={'index': 5, 'speaker': 'Jeff Bezos'}), 
        Document(text="And Jeff Bezos spent all Jeff Bezos's summers on that ranch from age four to 16.And Jeff Bezos's grandfather was really taking Jeff Bezos in the summers and theIn the early summers, Jeff Bezos's grandfather was letting Jeff Bezos pretend to help on the ranch because, of course, a four-year-old is a burden, not a help in real life.", metadata={'index': 6, 'speaker': 'Jeff Bezos'}),
        Document(text="Jeff Bezos's grandfather was really just watching Jeff Bezos and taking care of Jeff Bezos.Jeff Bezos's grandfather was doing that because Jeff Bezos's mom was so young.Jeff Bezos's mom had Jeff Bezos when Jeff Bezos's mom was 17, and so Jeff Bezos's grandfather was sort of giving her a break, and Jeff Bezos's grandmother and Jeff Bezos's grandfather would take Jeff Bezos for the summers.", metadata={'index': 7, 'speaker': 'Jeff Bezos'})]


In [4]:
class RetrieveFromGraph:
    def __init__(self, graph_name) -> None:
        self.db = FalkorDB(host='localhost', port=6379)
        self.graph = self.db.select_graph(graph_name)
        self.QUESTION_GEN_PROMPT = """
        You’re a Cypher expert, with access to the following graph:
        The knowledge graph schema is as follows:
        The graph contains the following node labels:
        {node_text}
        the Module label has {node_length} number of nodes.
        The graph contains the following relationship types:
        {relationship_text}
        This is the end of the knowledge graph schema description.
        Ensure to generate the cypher query for the question passed to you
        """
        self.question_prompt_template = PromptTemplate.from_template(self.QUESTION_GEN_PROMPT)

    def create_query(self, question):
        response = client.chat.completions.create(
        model="gpt-4-turbo",
        temperature=0.0,
        messages=[
            {"role": "system", "content": self.question_prompt_template.format(
                node_text=self.view_nodes(), 
                node_length=self.get_total_number_of_nodes(), 
                relationship_text=self.view_edges())},
            {"role": "user", "content": f"{question}"},
        ]
        )
        return response.choices[0].message.content

    def view_nodes(self):
        node_text= ""
        nodes = self.graph.query("""MATCH (n) RETURN n""")
        for node in nodes.result_set:
            node_text += str(node[0]) + "\n"
        return (node_text)
    
    def get_total_number_of_nodes(self):
        return len(self.graph.query("""MATCH (n) RETURN n""").result_set)
    
    def get_unique_edge_set(self):
        edge_set = set()
        edges = self.graph.query("""MATCH ()-[r]->() RETURN type(r) AS type, properties(r) AS properties""")
        for e in edges.result_set:
            temp_e = str(e[0])
            for key, value in e[1].items():
                temp_e += str(f" {{{key}: {value}}}")
            edge_set.add(temp_e)
        return edge_set
    
    def view_edges(self):
        relationship_text = ""
        edge_set = self.get_unique_edge_set()
        for e in edge_set:
            relationship_text += str(e)+ "\n"
        return (relationship_text)
    
    def get_cypher_text_from_output(self, text):
        pattern = r'```cypher\n(.*?)\n```'
        cypher_query = re.search(pattern, text, re.DOTALL)
        return cypher_query.group(1)
    
    def get_total_number_of_edges(self):
        edge_set = self.get_unique_edge_set()
        edge_len=len(edge_set)
        return edge_len

    def get_answer(self, question):
        query = self.create_query(question)
        print("QUERY:",query)
        cypher_query = self.get_cypher_text_from_output(query)
        answer=self.graph.query(cypher_query)
        return answer.result_set[0]

In [32]:
class TextToCypher:
    TRIPLET_PROMPT = """
                    You are an expert at creating graphs from text using OpenCypher.
                    You are provided with a text and your job is to create a graph from the text.
                    Ensure to create the graph based on the relationships between the entities in the text.
                    Ensure too set nouns as nodes and verbs as relationships.

                    ## Ensure to not use metadata unless necessary and ensure to normalize the relationships between the nodes.:
                    For example:
                    Incorrect:
                    ```
                    MERGE (p1:Person { name: "David", interest: ["Guitar"] })
                    MERGE (p2:Person { name: "Sarah", interest: ["Guitar"] })
                    ```
                    Correct(normalised relationships):
                    ```
                    MERGE (s:Interest { name: "Guitar" })
                    MERGE (p1:Person { name: "David" })
                    MERGE (p1)-[:HAS]->(s)
                    MERGE (p2:Person { name: "Sarah" })
                    MERGE (p2)-[:HAS]->(s)
                    ```

                    ##Ensure to capture possessive relationships as well:
                    John's car is red: should generate a node for John and a node for car with property color as red.
                    
                    ##Ensure to not be ambiguous in the relationships between the nodes:
                    Ensure to capture the relationships between the nodes with enough detail to not be ambiguous.

                    ##Ensure to Reify the relationships between the nodes:
                    To reify a relationship is to make it an entity in its own right. 
                    For example, if you have a relationship between a person and a car, 
                    you can reify that relationship into a new entity called "owns" and connect the person and the car to the "owns" entity.

                    ##Capture all details in the text within the graph:
                    """

    @classmethod
    def cypher_from_text_first_pass(cls, text):
        #TODO: entire text is dumped for the time being, need to split it into chunks if it exceeds the limit
        USER_PROMPT = """
                    The text is:
                    {text} 
                    Try to not use any metadata and ensure to normalize the relationships between the nodes, making all relations explicit.
                    Ensure the relationships between the nodes are not ambiguous and capture all details in the text within the graph and make sense.
                    """
        response = client.chat.completions.create(
        model="gpt-4-turbo",
        temperature=0.0,
        messages=[
            {"role": "system", "content": cls.TRIPLET_PROMPT},
            {"role": "user", "content": USER_PROMPT.format(text=text)},
        ]
        )
        return response.choices[0].message.content

    @classmethod
    def get_cypher_text_from_result(cls, text):
        pattern = r'```cypher\n(.*?)\n```'
        cypher_query = re.search(pattern, text, re.DOTALL)
        return cypher_query.group(1)

    @classmethod
    def cypher_from_text_second_pass(cls, text, cypher):
        USER_PROMPT = """
                        The text is: {text}. The cypher query is: {cypher}, modify the cypher query to even include implicit relationships between the nodes. 
                        Do not add in any nodes or relationships that are not present in the text. Ensure to capture all relationships between the nodes in the text.
                    """
        response = client.chat.completions.create(
        model="gpt-4-turbo",
        temperature=0.0,
        messages=[
            {"role": "system", "content": cls.TRIPLET_PROMPT},
            {"role": "user", "content": USER_PROMPT.format(text=text, cypher=cypher)},
        ]
        )
        return response.choices[0].message.content
    
    @classmethod
    def run(cls, text):
        first_pass = cls.cypher_from_text_first_pass(text)
        first_cypher = cls.get_cypher_text_from_result(first_pass)
        second_pass = cls.cypher_from_text_second_pass(text, first_cypher)
        second_cypher = cls.get_cypher_text_from_result(second_pass)
        #print("first cypher:", first_cypher)
        #print("second cypher:", second_cypher)
        return second_cypher


In [4]:
#initializing graph
graph_name = "Conv1"
db = FalkorDB(host='localhost', port=6379)
g = db.select_graph(graph_name)

In [39]:
i = 2
text = docs[i].text
print(text)

And Lex Fridman heard Jeff Bezos had a lot of work to do around the ranch.So what's the coolest job Jeff Bezos remembers doing there?


In [40]:
cypher = TextToCypher.run(text) 
print(cypher)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


// Create nodes for the entities
MERGE (lex:Person {name: "Lex Fridman"})
MERGE (jeff:Person {name: "Jeff Bezos"})
MERGE (work:Activity {name: "Work"})
MERGE (ranch:Location {name: "Ranch"})
MERGE (job:Activity {name: "Job"})

// Create relationships between nodes
MERGE (lex)-[:HEARD {about: "Work at Ranch"}]->(jeff)
MERGE (jeff)-[:HAD]->(work)
MERGE (work)-[:LOCATED_AT]->(ranch)
MERGE (jeff)-[:REMEMBERS]->(job)
MERGE (job)-[:ASSOCIATED_WITH]->(ranch)  // Implicit relationship as the job is related to the ranch activities


In [9]:
def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    return np.array(client.embeddings.create(input = [text], model=model).data[0].embedding)

def cosine_similarity(A, B):
    dot_product = np.dot(A, B)
    norm_A = np.linalg.norm(A)
    norm_B = np.linalg.norm(B)
    return dot_product / (norm_A * norm_B)

def create_metadata(input_string):
    pattern = r'(\w+):\s*"([^"]*)"|(\w+):\s*(\d+)'

    # Initialize the metadata dictionary
    metadata = {}

    # Find all key-value pairs using re.finditer
    matches = re.finditer(pattern, input_string)
    for match in matches:
        if match.group(1) and match.group(2):  # If the key-value pair is in the form of "key: "value""
            key = match.group(1)
            value = match.group(2)
        elif match.group(3) and match.group(4):  # If the key-value pair is in the form of "key: value"
            key = match.group(3)
            value = int(match.group(4))
        metadata[key] = value

    # Print the metadata dictionary
    return metadata

def cypher_to_json(cypher):
    node_dict = {}
    rel_dict = {}
    # Define regex patterns to match nodes and relationships
    node_pattern = r'MERGE \((\w+):(\w+) {([^}]+)}\)'
    relationship_pattern = r'MERGE \((\w+)\)-\[:(\w+)\]->\((\w+)\)'

    for match in re.finditer(node_pattern, cypher):
        list = match.groups()
        node_dict[list[0]] = {
            "type": list[1],
            "metadata": create_metadata(list[2])
        }

    for match in re.finditer(relationship_pattern, cypher):
        rel_dict[match.group(2)] = {
            "from": match.group(1),
            "to": match.group(3)
        }
    #use json.dumps to convert the dictionary to a JSON string
    return node_dict, rel_dict

#### DISPLAY NODES AND RELATIONS IN DICTIONARY FORMAT:

In [10]:
n,r = cypher_to_json(cypher)
print("NODES:",n)
print("RELATIONS:",r)

NODES: {'jeff': {'type': 'Person', 'metadata': {'name': 'Jeff Bezos'}}, 'lex': {'type': 'Person', 'metadata': {'name': 'Lex Fridman'}}, 'amazon': {'type': 'Company', 'metadata': {'name': 'Amazon'}}, 'blueOrigin': {'type': 'Company', 'metadata': {'name': 'Blue Origin'}}, 'conversation': {'type': 'Conversation', 'metadata': {'type': 'First long conversation'}}}
RELATIONS: {'FOUNDED': {'from': 'jeff', 'to': 'blueOrigin'}, 'PARTICIPATED_IN': {'from': 'lex', 'to': 'conversation'}, 'WILL_TALK_AGAIN': {'from': 'jeff', 'to': 'lex'}, 'OWNS': {'from': 'jeff', 'to': 'blueOrigin'}, 'INVOLVES': {'from': 'conversation', 'to': 'lex'}, 'ENJOYED': {'from': 'lex', 'to': 'conversation'}}


In [19]:
s_1 = ["Jeff bezos", "the ranch", "Lex Friedman", "jeff's grandfather", "Bezos"]
s_2 = ["Jeff"   , "ranch" , "Lex Fridman" , "grandfather", "Jeff Bezos"]

for i, j in zip(s_1, s_2):
    print(f"FUZZ RATIO between {i} and {j}:", fuzz.ratio(i, j))

FUZZ RATIO between Jeff bezos and Jeff: 57
FUZZ RATIO between the ranch and ranch: 71
FUZZ RATIO between Lex Friedman and Lex Fridman: 96
FUZZ RATIO between jeff's grandfather and grandfather: 76
FUZZ RATIO between Bezos and Jeff Bezos: 67


In [ ]:
print()

In [10]:
def extract_relationships_and_nodes_with_properties(cypher_str):
    # Regular expression patterns to match the CREATE statements with relationships and properties
    node_pattern = re.compile(r'\((\w+):(\w+)(?: \{([^\}]*)\})?\)')
    relationship_pattern = re.compile(r'MERGE \((\w+)\)-\[:(\w+)\]->\((\w+)\)')

    # Function to parse properties string to a dictionary
    def parse_properties(properties_str):
        if not properties_str:
            return {}
        properties = properties_str.split(', ')
        prop_dict = {}
        for prop in properties:
            key, value = prop.split(': ')
            # Remove quotes from string values
            value = value.strip('"')
            prop_dict[key] = value
        return prop_dict

    nodes = {}
    relationships = {}

    # Extract nodes and their properties
    for match in node_pattern.finditer(cypher_str):
        node_var, label, properties_str = match.groups()
        properties = parse_properties(properties_str)
        nodes[node_var] = {"label": label, "properties": properties}

    # Extract relationships and associate nodes
    for match in relationship_pattern.finditer(cypher_str):
        node1, rel_type, node2 = match.groups()
        if rel_type not in relationships:
            relationships[rel_type] = []
        relationships[rel_type].append({
            "node_1": nodes[node1],
            "node_2": nodes[node2]
        })

    return relationships

In [11]:
rel_dict = extract_relationships_and_nodes_with_properties(cypher)
print(rel_dict)

{'FOUNDED': [{'node_1': {'label': 'Person', 'properties': {'name': 'Jeff Bezos'}}, 'node_2': {'label': 'Organization', 'properties': {'name': 'Amazon'}}}, {'node_1': {'label': 'Person', 'properties': {'name': 'Jeff Bezos'}}, 'node_2': {'label': 'Organization', 'properties': {'name': 'Blue Origin'}}}], 'PARTICIPATED_IN': [{'node_1': {'label': 'Person', 'properties': {'name': 'Jeff Bezos'}}, 'node_2': {'label': 'Conversation', 'properties': {'kind': 'first time', 'length': 'long'}}}, {'node_1': {'label': 'Person', 'properties': {'name': 'Lex Fridman'}}, 'node_2': {'label': 'Conversation', 'properties': {'kind': 'first time', 'length': 'long'}}}], 'TOLD': [{'node_1': {'label': 'Person', 'properties': {'name': 'Jeff Bezos'}}, 'node_2': {'label': 'Person', 'properties': {'name': 'Lex Fridman'}}}], 'COULD_HAVE_TALKED_FOR_MORE_HOURS_WITH': [{'node_1': {'label': 'Person', 'properties': {'name': 'Jeff Bezos'}}, 'node_2': {'label': 'Person', 'properties': {'name': 'Lex Fridman'}}}], 'SURE_THAT_W

In [36]:
g.query(cypher)
nodes = g.query("""MATCH (n) RETURN n""")
for node in nodes.result_set:
    print(node[0])

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


first cypher: // Create nodes
MERGE (podcast:Podcast {name: "Lex Friedman Podcast"})
MERGE (jeff:Person {name: "Jeff Bezos"})
MERGE (grandfather:Person {name: "Grandfather"}) // Assuming the name is not specified
MERGE (ranch:Location {name: "Ranch", state: "Texas"})
MERGE (childhood:Period {name: "Childhood"})

// Create relationships
MERGE (jeff)-[:GUEST_ON]->(podcast)
MERGE (childhood)-[:SPENT_WITH]->(grandfather)
MERGE (childhood)-[:OCCURRED_AT]->(ranch)
MERGE (jeff)-[:HAD]->(childhood)
second cypher: // Create nodes
MERGE (podcast:podcast {name: "Lex Friedman Podcast"})
MERGE (jeff:person {name: "Jeff Bezos"})
MERGE (grandfather:person {name: "Grandfather"}) // Assuming the name is not specified
MERGE (ranch:location {name: "Ranch", state: "Texas"})
MERGE (childhood:period {name: "Childhood"})

// Create relationships
MERGE (jeff)-[:GUEST_ON]->(podcast)
MERGE (jeff)-[:SPENT]->(childhood)
MERGE (childhood)-[:WITH]->(grandfather)
MERGE (childhood)-[:AT]->(ranch)
MERGE (jeff)-[:HAD]-

In [37]:
g.query(cypher)

In [38]:
nodes = g.query("""MATCH (n) RETURN n""")
for node in nodes.result_set:
    print(node[0])

(:person{name:"Jeff Bezos"})
(:person{name:"Lex Fridman"})
(:company{name:"Amazon"})
(:company{name:"Blue Origin"})
(:conversation{firstTime:True,length:"long",type:"Interview"})
(:futureConversations{description:"Potential continuation"})
(:podcast{name:"Lex Friedman Podcast"})
(:person{name:"Grandfather"})
(:location{name:"Ranch",state:"Texas"})
(:period{name:"Childhood"})


In [47]:
net = Network(notebook=True, cdn_resources='in_line')

def node_exists(net, node_id):
    return any(node['id'] == node_id for node in net.nodes)

def save_graph_image(filename):
    net.show_buttons(filter_=['physics'])
    if filename.endswith('.html'):
        net.show(filename)
    else:
        print("Filename should have a .html extension")
    net.show(filename)

def create_graph_image(graph, filename):
    for chunk in graph:
        labels = [chunk.node_1.label, chunk.node_2.label]
        ids = [chunk.node_1.name, chunk.node_2.name]
        relationship = chunk.relationship
        print(f"{labels[0]}: {ids[0]} --> {labels[1]}:{ids[1]}; rel:{relationship}")

        for label,id in zip(labels,ids):
            if not node_exists(net, node_id=id):
                net.add_node(id, label=f"{label}:{id}")

        net.add_edge(ids[0], ids[1], title=relationship, arrows='to')

    save_graph_image(filename)


def extract_relationships_and_nodes_with_properties(cypher_str):
    # Regular expression patterns to match the CREATE statements with relationships and properties
    node_pattern = re.compile(r'\((\w+):(\w+)(?: \{([^\}]*)\})?\)')
    relationship_pattern = re.compile(r'CREATE \((\w+)\)-\[:(\w+)\]->\((\w+)\)')

    # Function to parse properties string to a dictionary
    def parse_properties(properties_str):
        if not properties_str:
            return {}
        properties = properties_str.split(', ')
        prop_dict = {}
        for prop in properties:
            key, value = prop.split(': ')
            # Remove quotes from string values
            value = value.strip('"')
            prop_dict[key] = value
        return prop_dict

    nodes = {}
    relationships = {}

    # Extract nodes and their properties
    for match in node_pattern.finditer(cypher_str):
        node_var, label, properties_str = match.groups()
        properties = parse_properties(properties_str)
        nodes[node_var] = {"label": label, "properties": properties}

    # Extract relationships and associate nodes
    for match in relationship_pattern.finditer(cypher_str):
        node1, rel_type, node2 = match.groups()
        if rel_type not in relationships:
            relationships[rel_type] = []
        relationships[rel_type].append({
            "node_1": nodes[node1],
            "node_2": nodes[node2]
        })

    return relationships

In [48]:
rel = extract_relationships_and_nodes_with_properties(cyp_text_2)
print(rel)

{'TOOK_CARE_OF_JB_DURING_SUMMERS': [{'node_1': {'label': 'Person', 'properties': {'name': "Jeff Bezos's Grandfather"}}, 'node_2': {'label': 'Person', 'properties': {'name': 'Jeff Bezos'}}}], 'INVOLVED': [{'node_1': {'label': 'Activity', 'properties': {'description': 'pretend to help on the ranch'}}, 'node_2': {'label': 'Person', 'properties': {'name': 'Jeff Bezos'}}}]}


In [17]:
create_graph_image(g, "graph.html")

TypeError: 'Graph' object is not iterable

In [19]:
print(graph)

NameError: name 'graph' is not defined

In [23]:
query = """
CREATE
(:Person {name:"Jeff Bezos"})-[:FOUNDED]->(:Organisation {name:"Amazon"}),
(:Person {name:"Jeff Bezos"})-[:FOUNDED]->(:Organisation {name:"Blue Origin"}),
(:Person {name:"Jeff Bezos"})-[:HAD_CONVERSATION_WITH {details: "could have talked for many more hours, will talk again", first_time: "true", feeling: "felt like"}]->(:Person {name:"Lex Fridman"}),
(:Person {name:"Lex Fridman"})-[:HAD_CONVERSATION_WITH {details: "could have talked for many more hours, will talk again", mutual_assurance: "true"}]->(:Person {name:"Jeff Bezos"}),
(:Person {name:"Jeff Bezos"})-[:SPENT_CHILDHOOD_IN {location: "ranch", details: "spent a lot of time, did a lot of work, remembers doing the coolest job"}]->(:Place {name:"Texas"}),
(:Person {name:"Jeff Bezos"})-[:SPENT_SUMMERS_WITH {age_range: "from age four to 16"}]->(:Person {name:"Jeff Bezos's grandfather"}),
(:Person {name:"Jeff Bezos"})-[:GRANDFATHER {relationship: "took care during summers, watched over, gave breaks to mom"}]->(:Person {name:"Jeff Bezos's grandfather"}),
(:Person {name:"Jeff Bezos's grandfather"})-[:TOOK_CARE_WITH]->(:Person {name:"grandmother"}),
(:Person {name:"Jeff Bezos"})-[:BORN_TO {age_of_mother: "17"}]->(:Person {name:"mom"});
"""
missing = CreateGraph.check_missing_nodes(query, doc[0].text)
print("MISSING NODES:",missing)

if missing.lower() != "none":
    modified_query = CreateGraph.fix_missing_nodes(query, doc[0].text, missing)
    print("FIXED QUERY:", modified_query)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


MISSING NODES: The provided cypher query captures several aspects of Jeff Bezos's life and interactions, including his founding of Amazon and Blue Origin, his conversation with Lex Fridman, and some details about his family and childhood. However, upon reviewing the text, it appears that the query does not include all the relevant information:

1. **Missing Relationship Details in Conversation**: The query mentions the conversation between Jeff Bezos and Lex Fridman, including some details about the conversation's nature and mutual feelings. However, it does not explicitly state that this was Jeff Bezos's first time doing a conversation of this kind and length, which is a significant aspect of the conversation's uniqueness.

2. **Missing Details about Lex Fridman's Assurance**: While the query mentions mutual assurance in the conversation, it specifically highlights that "Lex Fridman is sure Lex Fridman and Jeff Bezos will" talk again. This assurance from Lex Fridman's perspective is n

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


FIXED QUERY: CREATE
(:Person {name:"Jeff Bezos"})-[:FOUNDED]->(:Organisation {name:"Amazon"}),
(:Person {name:"Jeff Bezos"})-[:FOUNDED]->(:Organisation {name:"Blue Origin"}),
(:Person {name:"Jeff Bezos"})-[:HAD_CONVERSATION_WITH {details: "could have talked for many more hours, will talk again", first_time: "true", feeling: "felt like", first_of_kind_and_length: "true"}]->(:Person {name:"Lex Fridman"}),
(:Person {name:"Lex Fridman"})-[:HAD_CONVERSATION_WITH {details: "could have talked for many more hours, will talk again", mutual_assurance: "true", assured_future_conversation: "true"}]->(:Person {name:"Jeff Bezos"}),
(:Person {name:"Jeff Bezos"})-[:SPENT_CHILDHOOD_IN {location: "ranch", details: "spent a lot of time, did a lot of work, remembers doing the coolest job"}]->(:Place {name:"Texas"}),
(:Person {name:"Jeff Bezos"})-[:SPENT_SUMMERS_WITH {age_range: "from age four to 16"}]->(:Person {name:"Jeff Bezos's grandfather"}),
(:Person {name:"Jeff Bezos"})-[:GRANDFATHER {relationship: 

In [8]:
graph = CreateGraph.run(docs=doc, graph_name="Convo-1")


▶︎ GRAPH MAKER LOG - 2024-06-02 18:19:03 - INFO 
Document: 1



Using Model:  gpt-4-turbo


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER LOG - 2024-06-02 18:19:23 - INFO 
Trying JSON Parsing: 
[
    {
        "node_1": {"label": "Person", "name": "Jeff Bezos"},
        "node_2": {"label": "Organisation", "name": "Amazon"},
        "relationship": "Jeff Bezos is the founder of Amazon."
    },
    {
        "node_1": {"label": "Person", "name": "Jeff Bezos"},
        "node_2": {"label": "Organisation", "name": "Blue Origin"},
        "relationship": "Jeff Bezos is the founder of Blue Origin."
    },
    {
        "node_1": {"label": "Person", "name": "Jeff Bezos"},
        "node_2": {"label": "Person", "name": "Lex Fridman"},
        "relationship": "Jeff Bezos had a conversation with Lex Fridman."
    },
    {
        "node_1": {"label": "Person", "name": "Lex Fridman"},
        "node_2": {"label": "Document", "name": "Lex Friedman Podcast"},
        "relationship": "Lex Fridman hosts the Lex Friedman Podcast."
    

Graph creatd with total number of Edges: 10
FIRST CREATION QUERY: CREATE
(:Person {name:"Jeff Bezos"})-[:Jeff_Bezos_is_the_founder_of_Amazon]->(:Organisation {name:"Amazon"}),
(:Person {name:"Jeff Bezos"})-[:Jeff_Bezos_is_the_founder_of_Blue_Origin]->(:Organisation {name:"Blue Origin"}),
(:Person {name:"Jeff Bezos"})-[:Jeff_Bezos_had_a_conversation_with_Lex_Fridman]->(:Person {name:"Lex Fridman"}),
(:Person {name:"Lex Fridman"})-[:Lex_Fridman_hosts_the_Lex_Friedman_Podcast]->(:Document {name:"Lex Friedman Podcast"}),
(:Person {name:"Jeff Bezos"})-[:Jeff_Bezos_spent_a_lot_of_his_childhood_with_his_grandfather_on_a_ranch_in_Texas]->(:Place {name:"Texas"}),
(:Person {name:"Jeff Bezos"})-[:Jeff_Bezos_spent_his_summers_from_age_four_to_16_on_a_ranch_under_the_care_of_his_grandfather]->(:Person {name:"Jeff Bezos's grandfather"}),
(:Person {name:"Jeff Bezos's grandfather"})-[:Jeff_Bezoss_grandfather_took_care_of_him_during_the_summers,_allowing_him_to_pretend_to_help_on_the_ranch]->(:Person {

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


MODIFIED QUERY: CREATE
(:Person {name:"Jeff Bezos"})-[:FOUNDER_OF]->(:Organisation {name:"Amazon"}),
(:Person {name:"Jeff Bezos"})-[:FOUNDER_OF]->(:Organisation {name:"Blue Origin"}),
(:Person {name:"Jeff Bezos"})-[:HAD_CONVERSATION_WITH]->(:Person {name:"Lex Fridman"}),
(:Person {name:"Lex Fridman"})-[:HOSTS]->(:Document {name:"Lex Friedman Podcast"}),
(:Person {name:"Jeff Bezos"})-[:SPENT_CHILDHOOD_AT {location: 'ranch', state: 'Texas'}]->(:Place {name:"Texas"}),
(:Person {name:"Jeff Bezos"})-[:CARED_FOR_BY {age_range: 'four to 16', during: 'summers'}]->(:Person {name:"Jeff Bezos's grandfather"}),
(:Person {name:"Jeff Bezos's grandfather"})-[:CARED_FOR {activity: 'pretend help on ranch', during: 'summers'}]->(:Person {name:"Jeff Bezos"}),
(:Person {name:"Jeff Bezos's mom"})-[:HAD_AT_AGE {age: '17'}]->(:Person {name:"Jeff Bezos"}),
(:Person {name:"Jeff Bezos's grandfather"})-[:GAVE_BREAK_TO {during: 'summers'}]->(:Person {name:"Jeff Bezos's mom"}),
(:Person {name:"Jeff Bezos's grandmo

In [9]:
graph_to_query = RetrieveFromGraph(graph_name="Convo-1")
nodes  = graph_to_query.view_nodes()
edges = graph_to_query.view_edges()

In [13]:
question = "How old was Jeff Bezos when his grandfather took care of him?"
ans = graph_to_query.get_answer(question)
print(ans)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


QUERY: To find out the age range during which Jeff Bezos's grandfather took care of him, we can use the relationship type `CARED_FOR_BY` which includes an age range attribute. Here's the Cypher query to retrieve this information:

```cypher
MATCH (p:Person {name: "Jeff Bezos"})-[r:CARED_FOR_BY]->(g:Person {name: "Jeff Bezos's grandfather"})
RETURN r.age_range AS AgeRange
```

This query matches the nodes representing Jeff Bezos and his grandfather connected by the `CARED_FOR_BY` relationship and returns the age range during which his grandfather took care of him.
['four to 16']
